
# Contra dance command line

### Aim:
Draw board in command-line type visualisation which can show beginning and end of contra dance move defined by user.

### Notes:
* I will be using camelCase for functions and variable.
* Position variables are zero indexed and the y-axis is top to bottom.



In [21]:
# imports and setup

import numpy as np
from pprint import pprint as pp

In [162]:
# define classes

class Dancer(object):
    """ A sprite representing a dancer. Dancers have the following properties:
    
    Attributes:
        symbol: A character representing the dancer on the floor e.g. 'a', '1'.
        floor: The dance floor on which this dancer will be dancing.
        position: A two-integer array giving the dancer's position e.g. [2,2].
    
    """
    def __init__(self, symbol, position, floor): 
        self.symbol = symbol
        self.floor = floor
        self.position = position
        self.floor.addDancer(self)  # add dancer to dance floor


class DanceFloor(object):
    """A schematic of a dance floor. It has the following properties:
    
    Attributes:
        length: An integer giving the length of the dance floor
        width: An integer giving the width of the dance floor.
        floor: An array representing the dance floor.
        dancers: A dictionary with key-value pairs of dancers on the floor and their positions
        
    """
    
    
    def __init__(self, length, width=8):
        self.length = length
        self.width = width
        try:
            self.floor = np.full((length, width), "_", dtype=str)
        except TypeError:
            print("Dimension argument(s) for DanceFloor object must be integers.")
        self.dancers = dict()
        
        
    def showFloor(self):
        """Shows the current state of the dance floor"""
        for i in self.floor:
            print(i)
        print("")
            
            
    def addDancer(self, dancer):
        """Adds a dancer to the floor at their starting position"""
        # check symbol not being used by another dancer
        if dancer.symbol not in self.dancers:
            # check symbol is single character
            if len(dancer.symbol) == 1:
                try:               
                    # check for another dancer in the same position
                    if self.floor[dancer.position[1], dancer.position[0]] == "_":
                        # change floor marker to dancer's symbol and update dict of dancers
                        self.floor[dancer.position[1], dancer.position[0]] = dancer.symbol
                        self.dancers[dancer.symbol] = dancer.position 
                    else:
                        print("There is already a dancer at the position %s." % dancer.position)
                except IndexError:
                    print("The position %s is out of bounds." % dancer.position)
            else:
                print("Symbol for a dancer must be a single character.")
        else:
            print("The symbol '%s' is already being used for a dancer." % dancer.symbol)
        
        
    def removeDancer(self, symbol):
        """Removes a dancer from the floor, searching by symbol"""
        try:
            pos = self.dancers[symbol]
            self.floor[pos[1], pos[0]] = "_"
            del self.dancers[symbol]
        except KeyError:
            print("No dancer with that symbol. Currently, the dancers are: ", end="")
            for key in self.dancers.keys():
                print(key, end=" ")

In [163]:
# testing dance floor class
floor = DanceFloor(8)     
dancer = Dancer('1', [2,3], floor)
dancer = Dancer('2', [5,3], floor)
dancer = Dancer('3', [2,4], floor)
dancer = Dancer('4', [5,4], floor)
floor.showFloor()
pp(floor.dancers)

['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '1' '_' '_' '2' '_' '_']
['_' '_' '3' '_' '_' '4' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']
['_' '_' '_' '_' '_' '_' '_' '_']

{'1': [2, 3], '2': [5, 3], '3': [2, 4], '4': [5, 4]}
